<a href="https://colab.research.google.com/github/shstreuber/AI/blob/main/CS345_545_Week_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**PATH SEARCH ALGORITHMS**
In this workbook, you will learn about the various path search (or path optimization) algorithms:
1. **Breadth First**, which expands the search frontier stepwise from a center point into all directions equally
2. **Uniform Cost**, which attaches a "cost", i.e. a weight factor to each new step from the center point, so that frontier expansion is based on the cheapest path available, but still goes in all directions.
3. **A Star (or A *)**, which adds to Uniform Cost a heuristic (=Greedy) Best-First function in order to prioritize frontier expansion in one single direction rather than all directions.

The goal of all of these algorithms is to find the **optimal path** from one location to another.






# **Example Scenario**

Look at the graph below.
* What is the shortest way to GET from Chicago to Boston?
* What is the shortest way to DRIVE from Chicago to Boston?
* What is the cheapest way to GET from Chicago to Boston?

<img src = "https://allisonznliu.com/assets/images/posts/2020/search-algorithms/us_map_distances.png">

As you can see in the map above:
* To **GET** from Chicago to Boston, you would have to
   * **FLY across Lake Michigan from Chicago to Detroit** (283 miles) and then
   * **FLY across Lake Erie from Detroit to Buffalo** (256 miles) and then
   * **DRIVE from Buffalo to Syracuse to Boston** (150 miles + 312 miles).
   
 This route is **1,001 miles** long.
* To **DRIVE** from Chicago to Boston based on our available routes, you would have to
  * **DRIVE** from Chicago to Indianapolis (182 miles) to Columbus (176 miles) to Cleveland (144 miles) to Buffalo (189 miles) to Syracuse (150 miles) to Boston (312 miles) for a **grand total of 1,153 miles**.
  * **DRIVE** from ... nah, you do the math! What other land routes can you use?
  * **BUT WHAT IF** the highway from Chicago to Indianapolis is closed for construction?
  
<img src = "https://degemmill.com/content/uploads/2017/07/iStock-450571709.jpg">

We can all agree that flying may be faster than driving, but flying is also more expensive than driving. So, **what is most important to you?**
* Getting there **MORE QUICKLY**?
* Getting there **CHEAPER**?
* Visiting as many different states as possible because you're on **VACATION**?

These are the questions that inform which path search algorithm we use.




#**1. Breadth First Search**
Breadth First Search (BFS) is a graph traversal algorithm that explores all the neighbor nodes at the present depth before moving on to the nodes at the next depth level. It starts at the root node (= Chicago) and explores each neighbor node at the current depth level (= Detroit, Cleveland, Indianapolis) before moving to the next depth level (= Buffalo, Pittsburgh, Columbus). BFS uses a queue data structure to keep track of the nodes to be visited.

Here's a **high-level overview of BFS**:

1. Enqueue the root node into the queue.
2. While the queue is not empty, dequeue a node from the queue.
3. If the dequeued node has not been visited yet, mark it as visited and enqueue all its neighboring nodes into the queue.
4. Repeat steps 2-3 until the queue is empty.
BFS is often used to find the shortest path in unweighted graphs and to explore all nodes in a graph.

<hr>

Below is a simple Python implementation of BFS. This code performs BFS on a simple graph represented as an **ADJACENCY LIST**.

* An **ADJACENCY LIST** is a data structure used to represent a graph, where each vertex (or node) in the graph is associated with a list of its neighboring vertices. In other words, for each node in the graph, the adjacency list contains a list of all nodes that are directly connected to it by edges.

It starts the traversal from node 0 and prints each visited node during the traversal. You can modify the processing step to suit your specific application.

In [ ]:
from collections import defaultdict, deque

def bfs(graph, start):
    visited = set()              # Set to keep track of visited nodes
    queue = deque([start])       # Initialize a queue with the start node
    visited.add(start)           # Mark the start node as visited

    while queue:
        node = queue.popleft()   # Dequeue a node from the front of the queue
        print(node)              # Process the current node (e.g., print or store it)

        for neighbor in graph[node]:  # Iterate through neighbors of the current node
            if neighbor not in visited:
                visited.add(neighbor)  # Mark neighbor as visited
                queue.append(neighbor) # Enqueue neighbor for further exploration

# Example graph represented as an adjacency list
graph = defaultdict(list)
graph[0] = [1, 2]
graph[1] = [0, 3, 4]
graph[2] = [0, 5]
graph[3] = [1]
graph[4] = [1, 6]
graph[5] = [2]
graph[6] = [4]

The graph for this adjacency list looks like this:


```
         0
        / \
       1   2
      /|   |
     3 4   5
        |
        6

```



In [ ]:
# Perform BFS starting from node 0
bfs(graph, 0) # Here is the BFS algorithm

##**Exercise 1**
What does the adjacency list look like for our map in the scenario?
Use the code row below to:
* Build the graph
* Perform the Breadth First Search (bfs) on it.


What does the result tell you? Is the optimal route the **cheapest** route? Or is it the **fastest** route?

# **2. Uniform Cost Search**
Uniform Cost Search (UCS) is a graph traversal algorithm similar to Dijkstra's algorithm but optimized for graphs with non-negative edge weights (i.e. the route doesn't get cheaper; it only gets more expensive). UCS explores the graph in a breadth-first manner, but **prioritizing nodes with the lowest total cost to reach. It always selects the node with the lowest cumulative cost from the start node to the current node.**

* A **PRIORITY QUEUE** is an abstract data structure that operates similarly to a regular queue or stack, but with elements having an associated priority. In a priority queue, elements with higher (or lower, depending on the implementation) priority are dequeued before those with lower priority. Priority queues are often implemented using heaps, which are specialized binary trees that ensure efficient insertion, deletion, and retrieval of elements based on their priority. We use **heapq** to implement the priority queue.

Here's a high-level overview of UCS:

1. Initialize a priority queue with the start node and its cost (0).
2. While the priority queue is not empty:
  * Dequeue the node with the lowest cost.
  * If the dequeued node is the goal node, return the path.
  * Otherwise, expand the dequeued node and enqueue its neighbors with updated costs.
3. Repeat until the priority queue is empty.
UCS guarantees to find the shortest path from the start node to the goal node in a graph with non-negative edge weights.

Below is a simple Python implementation of Uniform Cost Search. This code performs UCS on a simple graph represented as an adjacency list. It starts the search from the start node and returns the total cost to reach the goal node. You can modify the graph and start/goal nodes to suit your specific scenario.

In [ ]:
import heapq

def ucs(graph, start, goal):
    pq = [(0, start)]  # Priority queue with (cost, node) tuples
    visited = set()    # Set to keep track of visited nodes

    while pq:
        cost, node = heapq.heappop(pq)  # Dequeue node with lowest cost
        if node == goal:
            return cost  # Return the total cost if goal is reached
        if node not in visited:
            visited.add(node)
            for neighbor, edge_cost in graph[node]:
                if neighbor not in visited:
                    heapq.heappush(pq, (cost + edge_cost, neighbor))  # Enqueue neighbors with updated cost

    return float('inf')  # Return infinity if no path is found

# Example graph represented as an adjacency list with (neighbor, edge_cost) tuples
graph = {
    'A': [('B', 1), ('C', 4)],
    'B': [('A', 1), ('C', 2), ('D', 5)],
    'C': [('A', 4), ('B', 2), ('D', 1)],
    'D': [('B', 5), ('C', 1)]
}


This graph looks like this:

```
       1       4
   A ----- B
   | \  2  / |
 1 |   \ / 5 | 1
   |   / \   |
   | /  1  \ |
   C ----- D
       2       5

```
In this visualization:

* Nodes A, B, C, and D represent vertices in the graph.
* Edges between nodes are labeled with their respective weights.
For example, there is an edge between nodes A and B with a weight of 1, an edge between nodes A and C with a weight of 4, and so on.
* This graph is **directed**, meaning that edges have a direction associated with them. However, the visualization does not explicitly show arrowheads to indicate directionality.

**NOTE** the cost numbers on the sides!


In [ ]:
# Computing the cost from start to goal
start_node = 'A'
goal_node = 'D'
total_cost = ucs(graph, start_node, goal_node) # Here is the UCS algorithm
print("Total cost from", start_node, "to", goal_node, "is", total_cost)

##**Exercise 2**
Now, try this using the two code fields below:
* Build **two adjacency lists** like the one in the UCS example for the map above.
  * The **FIRST ONE** is for FLYING and DRIVING from Chicago to Boston.
  * The **SECOND ONE** is for DRIVING ONLY from Chicago to Boston.
* The start_node is always Chicago. The goal_node is always Boston.
* Use the miles in the map in the Example Scenario above as the cost, but **ASSUME THAT FLYING is TWICE AS EXPENSIVE AS DRIVING**.
* Use **ucs** to calculate the cost for each route.

#**3. A* Search**
A* search is a graph traversal algorithm that efficiently finds the shortest path from a start node to a goal node by intelligently exploring nodes based on both the cost to reach them from the start node and an estimate of the cost to reach the goal node from them. It uses a heuristic function to guide the search, prioritizing nodes with the lowest total cost, which is a combination of the actual cost from the start node (g_cost) and the estimated cost to the goal node (h_cost).

**Here's a high-level overview of A* search**:

1. Initialize a priority queue with the start node and its total cost (g_cost + h_cost).
2. While the priority queue is not empty:
  * Dequeue the node with the lowest total cost.
  * If the dequeued node is the goal node, return the path.
  * Otherwise, expand the dequeued node and enqueue its neighbors with updated costs.
3. Repeat until the priority queue is empty.
A* search guarantees to find the shortest path from the start node to the goal node in graphs with non-negative edge weights, provided that the heuristic function is admissible (never overestimates the true cost) and consistent (satisfies the triangle inequality).

Below is a Python implementation of A* search. This code performs an A* search on a simple graph represented as an adjacency list. It starts the search from the start node and returns the total cost to reach the goal node. You can modify the graph and start/goal nodes to suit your specific scenario.

In [ ]:
import heapq

def astar(graph, start, goal):
    pq = [(0 + heuristic(start, goal), 0, start)]  # Priority queue with (total_cost, g_cost, node) tuples
    visited = set()  # Set to keep track of visited nodes

    while pq:
        _, cost, node = heapq.heappop(pq)  # Dequeue node with lowest total cost
        if node == goal:
            return cost  # Return the total cost if goal is reached
        if node not in visited:
            visited.add(node)
            for neighbor, edge_cost in graph[node]:
                if neighbor not in visited:
                    g_cost = cost + edge_cost  # Update g_cost
                    heapq.heappush(pq, (g_cost + heuristic(neighbor, goal), g_cost, neighbor))  # Enqueue with total cost

    return float('inf')  # Return infinity if no path is found

def heuristic(node, goal):
    # Example: Manhattan distance heuristic
    return abs(node[0] - goal[0]) + abs(node[1] - goal[1])

# Example graph represented as an adjacency list with (neighbor, edge_cost) tuples
graph = {
    (0, 0): [((0, 1), 1), ((1, 0), 1)],
    (0, 1): [((0, 0), 1), ((0, 2), 1)],
    (0, 2): [((0, 1), 1), ((1, 2), 1)],
    (1, 0): [((0, 0), 1), ((1, 1), 1)],
    (1, 1): [((1, 0), 1), ((1, 2), 1)],
    (1, 2): [((0, 2), 1), ((1, 1), 1)]
}


This graph looks like this:


```
(0, 0) --- (0, 1) --- (0, 2)
 |   \       |         |
 |    \1     |1        |1
 |     \     |         |
(1, 0) --- (1, 1) --- (1, 2)

```
In this visualization:

* Each node is represented by its coordinates (x, y).
* Edges between nodes are labeled with their respective weights.
For example, there is an edge between nodes (0, 0) and (0, 1) with a weight of 1, an edge between nodes (0, 0) and (1, 0) with a weight of 1, and so on.
* This graph is **undirected**, meaning that edges do not have a direction associated with them. Therefore, the visualization does not include arrowheads to indicate directionality.



In [ ]:
#Computing the cost from start to goal
start_node = (0, 0)
goal_node = (1, 2)
total_cost = astar(graph, start_node, goal_node) # Here is the A* algorithm
print("Total cost from", start_node, "to", goal_node, "is", total_cost)

##**Exercise 3**
Now, try this using the two code fields below:
* Build **the adjacency list** like the one in the A* example for the map above.
* The start_node is Chicago. The goal_node is Boston.
* Use the miles in the map in the Example Scenario above as the cost, but **ASSUME THAT FLYING is TWICE AS EXPENSIVE AS DRIVING**.
* Use **astar** to calculate the cost for each route.